In [5]:
from ultralytics import YOLO
from sklearn.metrics import f1_score
from PIL import Image
import pandas as pd
import statistics
import os

#Модуль для анализа модели на val(images/labesl) датасете, выводит mean IoU, median IoU, F1_macro по классам

In [6]:
def xywhn_to_xyxy(x_center, y_center, w, h, image_width, image_height):
    x_min = (x_center - w / 2) * image_width
    y_min = (y_center - h / 2) * image_height
    x_max = (x_center + w / 2) * image_width
    y_max = (y_center + h / 2) * image_height
    return x_min, y_min, x_max, y_max

def calculate_iou(box1, box2, image_width, image_height):
    # Преобразуем xywhn в xyxy
    x_min1, y_min1, x_max1, y_max1 = xywhn_to_xyxy(*box1, image_width, image_height)
    x_min2, y_min2, x_max2, y_max2 = xywhn_to_xyxy(*box2, image_width, image_height)
    
    # Найдем координаты пересечения
    inter_x_min = max(x_min1, x_min2)
    inter_y_min = max(y_min1, y_min2)
    inter_x_max = min(x_max1, x_max2)
    inter_y_max = min(y_max1, y_max2)
    
    # Вычислим площадь пересечения
    if inter_x_min < inter_x_max and inter_y_min < inter_y_max:
        intersection_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    else:
        intersection_area = 0
    
    # Вычисляем площади прямоугольников
    area1 = (x_max1 - x_min1) * (y_max1 - y_min1)
    area2 = (x_max2 - x_min2) * (y_max2 - y_min2)
    
    # Вычисляем IoU
    iou = intersection_area / (area1 + area2 - intersection_area)
    
    return iou

In [4]:
model = YOLO('model.pt')

iou_list = []
test_cls_list = []
pred_cls_list = []

images_path = 'test/images/'
labels_path = 'test/labels/'
for file_name in os.listdir(images_path):
    
    #Прогноз модели
    threshold = 0.4
    pred = model.predict(
        images_path+file_name,
        # save=True,
        conf=threshold,
        iou=0.4,
        max_det=5,
        classes=[0,1])[0]
    boxes = pred.boxes
    
    #Если объекта нет, ставим нули
    if len(boxes) >= 1:
        pred_x1, pred_y1, pred_x2, pred_y2 = boxes.xywhn[0].tolist()
        pred_cls = int(boxes.cls[0])
        pred_cls_list.append(pred_cls)
        conf = float(boxes.conf[0])
    else:
        pred_x1, pred_y1, pred_x2, pred_y2 = 0, 0, 0, 0
        pred_cls_list.append(0)
        conf = 0
    
    #Считываем labels
    with open(labels_path + file_name[:-4] + '.txt', 'r') as test_file:
        data = test_file.readline().split(' ')
        test_cls = float(data[0])
        test_cls_list.append(test_cls)
        test_x1, test_y1, test_x2, test_y2 = float(data[1]), float(data[2]), float(data[3]), float(data[4])
        
        image = Image.open(images_path+file_name)
        width, height = image.size

        iou = calculate_iou(
            [pred_x1, pred_y1, pred_x2, pred_y2], 
            [test_x1, test_y1, test_x2, test_y2], 
            width, height)
        
        iou_list.append(iou)

In [24]:
print('Mean: ' + str(statistics.mean(iou_list)))
print('Median: ' + str(statistics.median(iou_list)))
print('F1 macro: ' + str(f1_score(test_cls_list, pred_cls_list, average='macro')))